# [部分単語のトークン化](https://www.tensorflow.org/tutorials/tensorflow_text/subwords_tokenizer)

データセットからサブワード語彙を生成し、それらを使って`text.BertTokenizer`を語彙から構築する。

サブワードトークナイザの主な利点は、単語ベースのトークン化と文字ベースのトークン化の間に加筆することができる。

In [1]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [2]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

`tensorflow_text`は三つのサブワードスタイルのトークナイザを持っている。

- `text.BertTokenizer`: ハイレベルインターフェースのクラス。BERTのトークン分割アルゴリズムと`WordpiceTokenizer`を含む。文章を入力として受け取り、トークンIDを返す。
- `text.WordpieceTokenizer`: レベルの低いインターフェースのクラス。[WordPiece algorithm](https://www.tensorflow.org/tutorials/tensorflow_text/subwords_tokenizer#applying_wordpiece)のみ実装されている。呼び出す前にテキストを標準化して単語に分割する必要がある。入力として単語を受け取り、トークンIDを返す。
- `text.SentencepieceTokenizer`: より複雑な設定が必要。初期化メソッドは学習前の*sentencepiece*モデルを必要とする。構築のやり方は[sentencepiece repository](https://github.com/google/sentencepiece#train-sentencepiece-model)を参照。トークン化する際に、入力として文章を受け付ける。

ここでは、トップダウン方式で既存の単語群から*wordpiece vocabulary*を構築する。これは日本語、中国語、韓国語には適用できない（これらの言語のトークン化には`text.SentencepieceTokenizer`、`text.UnicodeCharTokenizer`、もしくは[このアプローチ](https://tfhub.dev/google/zh_segmentation/1)を使う）。

## データセットのダウンロード

ここではポルトガル語と英語の翻訳データセットをtfdsからダウンロードする。このデータセットは全て小文字で、句読点の周りにはスペースがあり、なんのユニコード標準化手法が使われているかは不明である。

In [3]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

In [4]:
for pt, en in train_examples.take(1):
    print("Portuguese: ", pt.numpy().decode('utf-8'))
    print("English:   ", en.numpy().decode('utf-8'))

Portuguese:  e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
English:    and when you improve searchability , you actually take away the one advantage of print , which is serendipity .


In [5]:
train_en = train_examples.map(lambda pt, en: en)
train_pt = train_examples.map(lambda pt, en: pt)

## ボキャブラリーを生成する

データセットから*wordpiece*ボキャブラリーを生成する。すでにボキャブラリーのファイルがある場合にはこのステップは必要ない。

In [6]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [7]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [8]:
%%time
pt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_pt.batch(1000).prefetch(2),
    **bert_vocab_args
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
CPU times: user 1min 53s, sys: 3.11 s, total: 1min 56s
Wall time: 1min 50s


In [9]:
print(pt_vocab[:10])
print(pt_vocab[100:110])
print(pt_vocab[1000:1010])
print(pt_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '#', '$', '%', '&', "'"]
['no', 'por', 'mais', 'na', 'eu', 'esta', 'muito', 'isso', 'isto', 'sao']
['90', 'desse', 'efeito', 'malaria', 'normalmente', 'palestra', 'recentemente', '##nca', 'bons', 'chave']
['##–', '##—', '##‘', '##’', '##“', '##”', '##⁄', '##€', '##♪', '##♫']


In [10]:
%%time
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_en.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 1min 25s, sys: 1.43 s, total: 1min 26s
Wall time: 1min 24s


In [11]:
print(en_vocab[:10])
print(en_vocab[100:110])
print(en_vocab[1000:1010])
print(en_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '#', '$', '%', '&', "'"]
['as', 'all', 'at', 'one', 'people', 're', 'like', 'if', 'our', 'from']
['choose', 'consider', 'extraordinary', 'focus', 'generation', 'killed', 'patterns', 'putting', 'scientific', 'wait']
['##_', '##`', '##ย', '##ร', '##อ', '##–', '##—', '##’', '##♪', '##♫']


ボキャブラリーファイルを作成する。

In [12]:
def write_vocab_file(filepath, vocab):
    with open(filepath, 'w') as f:
        for token in vocab:
            print(token, file=f)

In [16]:
write_vocab_file('pt_vocab.txt', pt_vocab)
write_vocab_file('en_vocab.txt', en_vocab)

In [17]:
ls *.txt

en_vocab.txt  pt_vocab.txt


## トークナイザの構築

`text.BertTokenizer`インスタンスはボキャブラリーのファイルパスを第一引数として渡すことで初期化できる。

In [18]:
pt_tokenizer = text.BertTokenizer('pt_vocab.txt', **bert_tokenizer_params)
en_tokenizer = text.BertTokenizer('en_vocab.txt', **bert_tokenizer_params)

これらのトークナイザを使って、テキストをエンコードできる。例として、英語のデータから三つの例文をバッチする。

In [19]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
    for ex in en_examples:
        print(ex.numpy())

b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'
b'but what if it were active ?'
b"but they did n't test for curiosity ."


これを`BertTokenizer.tokenize`メソッドを通して実行してみる。デフォルトでは、`(batch, word, word-piece)`の形をした`tf.RaggedTensor`を返す。

In [20]:
# Tokenize the examples -> (batch, word, word-piece)
token_batch = en_tokenizer.tokenize(en_examples)
# Merge the word and word-piece axes -> (batch, tokens)
token_batch = token_batch.merge_dims(-2,-1)

for ex in token_batch.to_list():
    print(ex)

[72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74, 2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15]
[87, 90, 107, 76, 129, 1852, 30]
[87, 83, 149, 50, 9, 56, 664, 85, 2512, 15]


トークンIDをテキスト表現に置き換えた場合（`tf.gather`を使う）、`"searchability"`は`search ##ability`に、`"serendipity"`は`"s ##ere ##nd ##ip ##ity"`に分解されている。

In [21]:
# Lookup each token id in the vocabulary.
txt_tokens = tf.gather(en_vocab, token_batch)
# Join with spaces.
tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'and when you improve search ##ability , you actually take away the one advantage of print , which is s ##ere ##nd ##ip ##ity .',
       b'but what if it were active ?',
       b"but they did n ' t test for curiosity ."], dtype=object)>

## カスタマイズとエクスポート

トークナイザとデトークナイザを構築し、それらをエクスポートする。

`reserved_tokens`はボキャブラリーの始めにある空白を予約する。したがって、`[START]`と`[END]`は二つの言語で同じインデックスを持つ。

In [25]:
reserved_tokens

['[PAD]', '[UNK]', '[START]', '[END]']

In [27]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
    count = ragged.bounding_shape()[0]
    starts = tf.fill([count,1], START)
    ends = tf.fill([count,1], END)
    return tf.concat([starts, ragged, ends], axis=1)

トークナイザをエクスポートする前に次のものをクリーンアップすることができる：

1. `[START]`や`[END]`、`[PAD]`などの予約トークンを削除することができる。
2. 結果の`words`軸をもとに文字列を結合できる。

In [28]:
def cleanup_text(reserved_tokens, token_txt):
    # Drop the reserved tokens, except for "[UNK]".
    bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
    bad_token_re = "|".join(bad_tokens)

    bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
    result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

    # Join them into strings.
    result = tf.strings.reduce_join(result, separator=' ', axis=-1)

    return result

次に、`text.BertTokenizer`インスタンスとカスタム論理、`@tf.function`ラッパーを含む`CustomTokenizer`クラスを構築する。

In [33]:
class CustomTokenizer(tf.Module):
    def __init__(self, reserved_tokens, vocab_path):
        self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
        self._reserved_tokens = reserved_tokens
        self._vocab_path = tf.saved_model.Asset(vocab_path)

        vocab = pathlib.Path(vocab_path).read_text().splitlines()
        self.vocab = tf.Variable(vocab)

        ## Create the signatures for export:   

        # Include a tokenize signature for a batch of strings. 
        self.tokenize.get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string))

        # Include `detokenize` and `lookup` signatures for:
        #   * `Tensors` with shapes [tokens] and [batch, tokens]
        #   * `RaggedTensors` with shape [batch, tokens]
        self.detokenize.get_concrete_function(
            tf.TensorSpec(shape=[None, None], dtype=tf.int64))
        self.detokenize.get_concrete_function(
            tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

        self.lookup.get_concrete_function(
            tf.TensorSpec(shape=[None, None], dtype=tf.int64))
        self.lookup.get_concrete_function(
            tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

        # These `get_*` methods take no arguments
        self.get_vocab_size.get_concrete_function()
        self.get_vocab_path.get_concrete_function()
        self.get_reserved_tokens.get_concrete_function()

    @tf.function
    def tokenize(self, strings):
        enc = self.tokenizer.tokenize(strings)
        # Merge the `word` and `word-piece` axes.
        enc = enc.merge_dims(-2,-1)
        enc = add_start_end(enc)
        return enc

    @tf.function
    def detokenize(self, tokenized):
        words = self.tokenizer.detokenize(tokenized)
        return cleanup_text(self._reserved_tokens, words)

    @tf.function
    def lookup(self, token_ids):
        return tf.gather(self.vocab, token_ids)

    @tf.function
    def get_vocab_size(self):
        return tf.shape(self.vocab)[0]

    @tf.function
    def get_vocab_path(self):
        return self._vocab_path

    @tf.function
    def get_reserved_tokens(self):
        return tf.constant(self._reserved_tokens)

それぞれの言語について`CustomTokenizer`を構築する。

In [34]:
tokenizers = tf.Module()
tokenizers.pt = CustomTokenizer(reserved_tokens, 'pt_vocab.txt')
tokenizers.en = CustomTokenizer(reserved_tokens, 'en_vocab.txt')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


AttributeError: in user code:

    <ipython-input-33-df4da815e1c7>:44 detokenize  *
        words = self.tokenizer.detokenize(tokenized)

    AttributeError: 'BertTokenizer' object has no attribute 'detokenize'
